In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from urllib.parse import urljoin

In [13]:
# Function to get products from all pages
def get_all_products():
    base_url = "https://api.nike.com"
    initial_url = "/discover/product_wall/v1/marketplace/US/language/en/consumerChannelId/d9a5bc42-4b9c-4976-858a-f159cf99c647"
    params = {
        "path": "/w/mens-shoes-nik1zy7ok",
        "attributeIds": "16633190-45e5-4830-a068-232ac7aea82c,0f64ecc7-d624-4e91-b171-b83a03dd8550",
        "queryType": "PRODUCTS",
        "anchor": "1",
        "count": "24"
    }

    # Headers
    headers = {
        'nike-api-caller-id': 'nike:dotcom:browse:wall.client:2.0',
        'anonymousid': '82C2FC43F58856356902F7058D12F873',
        'cookie': 'geoloc=cc=NG,...',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
        'accept': '*/*',
        'accept-encoding': 'gzip, deflate, br, zstd',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'origin': 'https://www.nike.com',
        'referer': 'https://www.nike.com/',
    }

    # List to store all products

    url = urljoin(base_url, initial_url)  # Construct the full URL

    df = pd.DataFrame(columns=['title', 'subtitle', 'productType', 'productSubType', 'color', 'url', 'imageUrl', 'currentPrice', 'discountPercentage', 'initialPrice'])
    while url:  # Continue until there are no more pages (url will be None when finished)
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = json.loads(response.text)  # Parse the JSON response
            for grouping in data['productGroupings']:
                for product in grouping['products']:
                    title = product['copy']['title'] if 'copy' in product and 'title' in product['copy'] else None
                    subtitle = product['copy']['subTitle'] if 'copy' in product and 'subTitle' in product['copy'] else None
                    productType = product['productType'] if 'productType' in product else None
                    productSubtype = product['productSubType'] if 'productSubtype' in product else None
                    color = product['displayColors']['simpleColor']['label'] if 'displayColors' in product and 'simpleColor' in product['displayColors'] and 'label' in product['displayColors']['simpleColor'] else None
                    url = product['pdpUrl']['url'] if 'pdpUrl' in product and 'url' in product['pdpUrl'] else None
                    imageUrl = product['colorwayImages']['portraitURL'] if 'colorwayImages' in product and 'portraitURL' in product['colorwayImages'] else None
                    currentPrice = product['prices']['currentPrice'] if 'prices' in product and 'currentPrice' in product['prices'] else None
                    discountPercentage = product['prices']['discountPercentage'] if 'prices' in product and 'discountPercentage' in product['prices'] else None
                    intialPrice = product['prices']['initialPrice'] if 'prices' in product and 'initialPrice' in product['prices'] else None
                    df = pd.concat([df, pd.DataFrame({'title':[title], 'subtitle': [subtitle], 'productType' : [productType], 'productSubType' : [productSubtype], 'color' : [color], 'url': [url], 
                                                      'imageUrl' : [imageUrl], 'currentPrice' : [currentPrice], 'discountPercentage' : [discountPercentage], 'initialPrice' : [intialPrice]})], ignore_index=True)
            # Check if there is a next page
            next_page = data.get('pages', {}).get('next', None)
            if next_page:
                url = urljoin(base_url, next_page)  # Join the base URL with the next URL
            else:
                url = None  # No more pages, exit the loop
        else:
            print(f"Error: {response.status_code}")
            break

    return df




In [14]:
# Fetch all products
nike_products = get_all_products()

C:\Users\win10\AppData\Local\Temp\ipykernel_9284\3139010228.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({'title':[title], 'subtitle': [subtitle], 'productType' : [productType], 'productSubType' : [productSubtype], 'color' : [color], 'url': [url],


In [15]:
nike_products.to_csv('scraped_products.csv')